#**Toxic Release Inventories**
Objective: To identify states in the US showing a higher release of toxic chemicals for the year 2023. Additionally to find out which industrial sectors contribute the most and least?

# RPAD 676: Data Science for the Public Good

## Author: Shristi Das




In [2]:
# Import libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load and clean data
df = pd.read_csv("2023_Cleaned TRI data.csv")
df.columns = ['Latitude', 'Longitude', 'Facility', 'State', 'Industry', 'Total_Releases']

# Dropping missing values
df = df.dropna(subset=['Latitude', 'Longitude', 'Total_Releases'])

# Check columns
df.columns
print(df)

# Convert numeric fields
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df['Total_Releases'] = pd.to_numeric(df['Total_Releases'], errors='coerce')

        Latitude   Longitude                                 Facility State  \
0      32.778670  -96.862760                                 BMIC LLC    TX   
1      39.664033  -75.807412                W L GORE & ASSOCIATES INC    MD   
2      39.683840  -84.275660                         VEOLIA N.A. INC.    OH   
3      42.884140  -70.886720                   HENKEL OF AMERICA INC.    NH   
4      32.558990  -92.941410                 HAYNES INTERNATIONAL INC    LA   
...          ...         ...                                      ...   ...   
77959  33.972290  -83.387650  PILGRIM'S PRIDE ATHENS PROCESSING PLANT    GA   
77960  37.742300  -85.892700                     MOUSER CABINETRY LLC    KY   
77961  37.979880 -100.840600   DAIRY FARMERS OF AMERICA - GARDEN CITY    KS   
77962  31.808090 -101.559070                      DEADWOOD CRYO PLANT    TX   
77963  42.499406  -94.017177            KOCH FERTILIZER FT. DODGE LLC    IA   

                     Industry  Total_Releases  
0  

In [3]:
# Aggregate spatial trends
state_totals = df.groupby('State')['Total_Releases'].sum().reset_index()
industry_totals = df.groupby('Industry')['Total_Releases'].sum().reset_index()

# Examine unique states
df.groupby('State').size().sort_values(ascending=False)

# Create a DataFrame
data = {
    'State': ['TX', 'OH', 'CA', 'PA', 'IN', 'LA', 'MI', 'WI'],
    'Report Count': [9506, 4779, 3482, 3381, 3275, 2966, 2950, 2671]
}

state_counts_df = pd.DataFrame(data)

# Display the table
state_counts_df

,State,Report Count
0,TX,9506
1,OH,4779
2,CA,3482
3,PA,3381
4,IN,3275
5,LA,2966
6,MI,2950
7,WI,2671


In [4]:
# Sort for top and bottom 5
top_states = state_totals.sort_values(by='Total_Releases', ascending=False).head()
top_industries = industry_totals.sort_values(by='Total_Releases', ascending=False).head()
bottom_industries = industry_totals.sort_values(by='Total_Releases', ascending=True).head()


# Spatial Mapping Using Folium

In [5]:
# Create base map centered on the U.S.
tri_map = folium.Map(location=[39.8283, -98.5795], zoom_start=5)
marker_cluster = MarkerCluster().add_to(tri_map)

# Limit to top 500 for performance
top500 = df.sort_values(by='Total_Releases', ascending=False).head(500)

# Add simple markers without popups or icons
for _, row in top500.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']]
    ).add_to(marker_cluster)

# Display the map in the notebook
tri_map

In [6]:
# To display in a tabular format
from IPython.display import display

# Converting abbreviations to full name of the states for easier readablity
state_name_map = {
    'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AZ': 'Arizona',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia',
    'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii',
    'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts',
    'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota',
    'MO': 'Missouri', 'MS': 'Mississippi', 'MT': 'Montana', 'NC': 'North Carolina',
    'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
    'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas',
    'UT': 'Utah', 'VA': 'Virginia', 'VT': 'Vermont', 'WA': 'Washington',
    'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'
}

top_states['State'] = top_states['State'].map(state_name_map)

# Printing results and values
print("Top 5 States by Total Releases:")
display(top_states)

print("\nTop 5 Industry Sectors by Total Releases:")
display(top_industries)

print("\nBottom 5 Industry Sectors by Total Releases:")
display(bottom_industries)

Top 5 States by Total Releases:


,State,Total_Releases
0,Alaska,8.995967e+08
48,Utah,2.857906e+08
47,Texas,2.528504e+08
36,Nevada,2.162987e+08
20,Louisiana,1.223614e+08



Top 5 Industry Sectors by Total Releases:


,Industry,Total_Releases
13,Metal Mining,1.501655e+09
2,Chemicals,4.240669e+08
22,Primary Metals,3.118488e+08
15,Natural Gas Processing,1.971212e+08
5,Electric Utilities,1.948628e+08



Bottom 5 Industry Sectors by Total Releases:


,Industry,Total_Releases
24,Publishing,0.14
25,Textile Product,155402.07
27,Tobacco,669283.11
23,Printing,1085081.74
11,Leather,1208311.14
